In [13]:
import random

class Settings:
    COLORS = {
        "BLEU": '\033[34m',
        "ROUGE": '\033[31m',
        "JAUNE": '\033[33m',
        "VERT": '\033[32m',
    }
    
    NB_PLUS_4 = 4
    NB_PLUS_2_PER_COLOR = 2
#     NB_PLUS_INV_PER_COLOR = 2
    NB_PLUS_STOP_PER_COLOR = 2
    NB_ZERO_PER_COLOR = 1
    NB_OTHER_PER_COLOR = 2
    MAX_VALUES_CARDS = 9
    
    NB_PLAYER = 2
    NB_CARDS_PER_PLAYER = 7
    
class Card:
    def __init__(self, color, value):
        self.color = color
        self.value = value
    
class Player:
    def __init__(self, pseudo):
        self.pseudo = pseudo
        self.main = []
        
    def addCard(self, card):
        self.main.append(card)
    
    def printMain(self):
        i = 1
        for card in self.main:
            print(card.color, str(i) + " - " + card.value)
            i+=1
        print("P - PIOCHER UNE CARTE")
        
class Game:
    def __init__(self, lstPlayer, lstCard):
        self.lstPlayer = lstPlayer
        self.lstCard = lstCard
        self.lastCardPlayed = None
        self.finish = False
        self.winner = None
        self.nextBlocked = False
        self.currentPlayer = None
        self.initialisation()
        
    def initialisation(self):
        random.shuffle(self.lstCard)        
        for i in range(Settings.NB_CARDS_PER_PLAYER):
            for player in self.lstPlayer:
                player.addCard(self.lstCard[0])  
                self.lstCard.pop(0)
                
        self.lastCardPlayed = self.lstCard[0]
        while self.lastCardPlayed.value == "+4":
            self.lstCard.pop(0)
            self.lastCardPlayed = self.lstCard[0]
    
    def printLstCard(self):
        for card in self.lstCard:
            print(card.color, card.value)
            
    def verifCardValide(self, card):
        if card.value == self.lastCardPlayed.value or card.color == self.lastCardPlayed.color or card.value == "+4":
            return True
        else:
            return False
        
    def piocher(self, player, nbCards):
        for i in range(nbCards):
            player.addCard(self.lstCard[0])  
            self.lstCard.pop(0)
    
            
    def applyEffectCard(self, card, player):
        if card.value == "stop":
            self.nextBlocked = True
        if card.value == "+2":
            self.nextBlocked = True
            self.piocher(player, 2)
        if card.value == "+4":
            self.nextBlocked = True
            print("Liste des couleurs :")
            for color in Settings.COLORS:
                print(Settings.COLORS[color], color)

            color = None
            while color is None:
                colorNumber = input("Choisissez la prochaine couleur : ")
                try:
                    color = Settings.COLORS[colorNumber]
                except:
                    print("Cette couleur n'existe pas !")
            card.color = color
            self.piocher(player, 4)
        self.lastCardPlayed = card
        
    def isWinner(self, player):
        if len(player.main) <= 0:
            self.winner = player
            self.finish = True
            print(player.pseudo + " à gagné !")
            
    def noMoreCards(self):
        if len(self.lstCard) <= 0:
            self.finish = True
            print("La partie est terminée, il n'y a plus de cartes ! EGALITEEEEEEEE !")
            
    def start(self):
        while not self.finish:
            i = 0
            for player in self.lstPlayer:
                self.currentPlayer = player
                if not self.nextBlocked:                    
                    print("Au tour de :", player.pseudo)
                    player.printMain()
                    valideCard = False
                    card = None

                    print(self.lastCardPlayed.color, "Dernière carte jouée : " + self.lastCardPlayed.value)
                    while not valideCard:
                        numCard = input("Choisissez le numéro d'une carte à poser : ")
                        if numCard == "P":
                            self.piocher(player, 1)
                            valideCard = True
                        else:                                                
                            try:
                                card = player.main[int(numCard)-1]
                                valideCard = self.verifCardValide(card)

                                if not valideCard:
                                    print("Vous ne pouvez pas jouer cette carte !")
                                else:
                                    nextPlayer = None
                                    if i+1 > len(self.lstPlayer)-1:
                                        nextPlayer = self.lstPlayer[0]
                                    else:
                                        nextPlayer = self.lstPlayer[i+1]
                                    self.applyEffectCard(card, nextPlayer)
                                    player.main.pop(int(numCard)-1)
                            except:
                                print("Vous ne pouvez pas jouer cette carte !")                                
                else:
                    print(player.pseudo + " a été bloqué !")
                    self.nextBlocked = False
                i+=1
                self.isWinner(player)
            self.noMoreCards()
        
def generateListCard():
    lstCard = []
    for i in range(Settings.NB_PLUS_4):
        lstCard.append(Card("\033[97m", "+4"))
    
    for color in Settings.COLORS:
        for i in range(Settings.NB_PLUS_2_PER_COLOR):
            lstCard.append(Card(Settings.COLORS[color], "+2"))
#         for i in range(Settings.NB_PLUS_INV_PER_COLOR):
#             lstCard.append(Card(Settings.COLORS[color], "inv"))
        for i in range(Settings.NB_PLUS_STOP_PER_COLOR):
            lstCard.append(Card(Settings.COLORS[color], "stop"))
        
        for i in range(Settings.NB_ZERO_PER_COLOR):            
            lstCard.append(Card(Settings.COLORS[color], "0"))
            
        for i in range(1, Settings.MAX_VALUES_CARDS+1): 
            for j in range(Settings.NB_OTHER_PER_COLOR):            
                lstCard.append(Card(Settings.COLORS[color], str(i)))
            
    return lstCard
    
def generateListPlayer():
    lstPlayer = []
    for i in range(1, Settings.NB_PLAYER+1):
        pseudo = input("Pseudo du joueur " + str(i) + " :")
        lstPlayer.append(Player(pseudo))
            
    return lstPlayer

game = Game(generateListPlayer(), generateListCard())
game.start()

Pseudo du joueur 1 : Louis
Pseudo du joueur 2 : Juif


Au tour de : Louis
 1 - 9
 2 - +2
 3 - 6
 4 - 7
 5 - +2
 6 - 5
 7 - 9
P - PIOCHER UNE CARTE
 Dernière carte jouée : 4


Choisissez le numéro d'une carte à poser :  P


Au tour de : Juif
 1 - 9
 2 - 8
 3 - 6
 4 - 3
 5 - 1
 6 - 7
 7 - 0
P - PIOCHER UNE CARTE
 Dernière carte jouée : 4


Choisissez le numéro d'une carte à poser :  P


...

Au tour de : Louis
 1 - 9
 2 - +2
 3 - 6
 4 - 7
 5 - +2
 6 - 5
 7 - 9
 8 - 4
 9 - 5
 10 - 5
 11 - 8
 12 - 5
 13 - stop
 14 - +2
 15 - 3
 16 - 6
 17 - 1
 18 - +4
 19 - 8
 20 - 1
 21 - 5
 22 - 8
 23 - 2
 24 - stop
 25 - 2
 26 - 8
 27 - 3
 28 - 9
 29 - 4
 30 - 3
 31 - 5
 32 - stop
 33 - +2
 34 - stop
 35 - 1
 36 - 3
 37 - 6
 38 - +4
 39 - 1
 40 - 4
 41 - 3
 42 - +2
 43 - 7
 44 - 1
 45 - 4
 46 - 5
 47 - +4
P - PIOCHER UNE CARTE
 Dernière carte jouée : 4


Choisissez le numéro d'une carte à poser :  P


Au tour de : Juif
 1 - 9
 2 - 8
 3 - 6
 4 - 3
 5 - 1
 6 - 7
 7 - 0
 8 - 6
 9 - 2
 10 - 3
 11 - 6
 12 - 0
 13 - 3
 14 - 8
 15 - 8
 16 - 0
 17 - stop
 18 - 9
 19 - stop
 20 - stop
 21 - 4
 22 - 7
 23 - 7
 24 - 2
 25 - 2
 26 - 5
 27 - +2
 28 - 4
 29 - 9
 30 - 1
 31 - 0
 32 - +2
 33 - 9
 34 - 2
 35 - 4
 36 - 1
 37 - 7
 38 - 2
 39 - 9
 40 - 7
 41 - 4
 42 - +2
 43 - 6
 44 - 2
 45 - 7
 46 - 8
 47 - 6
P - PIOCHER UNE CARTE
 Dernière carte jouée : 4


Choisissez le numéro d'une carte à poser :  P


La partie est terminée, il n'y a plus de cartes ! EGALITEEEEEEEE !
